In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt

filename = '/content/drive/MyDrive/Coding Dojo/Semana 5/kc_house_data.csv'
df = pd.read_csv(filename)
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [5]:
df["date"]=df.loc[:,"date"].str.slice(stop=4).astype(float)

#df["date"]=pd.to_datetime(df['date'])

In [6]:
from sklearn.model_selection import train_test_split
X=df.drop(["id","price"], axis=1).values
y=df["price"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [7]:
# Creamos un objeto (una variable) a partir del regresor de árbol de decisión
reg = DecisionTreeRegressor(max_depth=20)
# Entrenamos nuestro modelo!
reg.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=20)

In [8]:
reg.score(X_test, y_test)

0.7381697870531001

In [9]:
df["price"].value_counts(normalize=True) #nuestras predicciones base son bajas

350000.0     0.007958
450000.0     0.007958
550000.0     0.007357
500000.0     0.007033
425000.0     0.006940
               ...   
607010.0     0.000046
1362500.0    0.000046
298800.0     0.000046
957500.0     0.000046
402101.0     0.000046
Name: price, Length: 4028, dtype: float64

In [10]:
from sklearn.ensemble import BaggingRegressor

bag_clf = BaggingRegressor(
    DecisionTreeRegressor(), n_estimators=100,
    bootstrap=True
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(y_pred)
print(y_test)
bag_clf.score(X_test, y_test)

[ 367856.5   883517.5  1117322.23 ...  333098.51  951552.3   498776.4 ]
[ 365000.  865000. 1038000. ...  410500.  940000.  410000.]


0.8582698262154655

In [11]:
from sklearn.ensemble import RandomForestRegressor

rndf_clf = RandomForestRegressor(n_estimators=100)
rndf_clf.fit(X_train, y_train)
y_pred = rndf_clf.predict(X_test)

print(y_pred)
print(y_test)
rndf_clf.score(X_test, y_test)

[ 375140.    847268.5  1120788.3  ...  325025.4   967917.1   482128.15]
[ 365000.  865000. 1038000. ...  410500.  940000.  410000.]


0.861747602983136

In [12]:
rndf_clf.feature_importances_

array([0.00211475, 0.00294258, 0.01031755, 0.2603653 , 0.01439398,
       0.00186223, 0.02779875, 0.01007419, 0.00297946, 0.33444324,
       0.01993899, 0.00527348, 0.03824099, 0.00201207, 0.01363812,
       0.14383516, 0.06552043, 0.03176556, 0.01248317])

In [13]:
features=df.drop(["id","price"], axis=1)
features.columns[3]

'sqft_living'

In [15]:
##¿Cuáles son las características más importantes de tu modelo?

#-Según mi modelo, las caracteristicas mas importantes son grade y sqft_living, lo que hace total sentido, 
#ya que al investigar, la columna grade es un ranking de la calidad de la vivienda y la columna sqft_living son los metros cuadrados 

##¿Qué otros parámetros podrías haber intentado ajustar?

#Hay un monton de hiperparametros que se podrian ajustar aun y se podria jugar con ellos y el numero de arboles, el tamaño de los arboles, etc.

id - Unique ID for each home sold
date - Date of the home sale
price - Price of each home sold
bedrooms - Number of bedrooms
bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
sqft_living - Square footage of the apartments interior living space
sqft_lot - Square footage of the land space
floors - Number of floors
waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not
view - An index from 0 to 4 of how good the view of the property was
condition - An index from 1 to 5 on the condition of the apartment,
grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
sqft_above - The square footage of the interior housing space that is above ground level
sqft_basement - The square footage of the interior housing space that is below ground level
yr_built - The year the house was initially built
yr_renovated - The year of the house’s last renovation
zipcode - What zipcode area the house is in
lat - Lattitude
long - Longitude
sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors